In [1]:
!wget --no-check https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta
    
    


--2019-12-29 11:55:20--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 13.226.52.155, 13.226.52.106, 13.226.52.168, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|13.226.52.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 810105 (791K) [application/octet-stream]
Saving to: ‘chr1.GRCh38.excerpt.fasta’

chr1.GRCh38.excerpt 100%[===================>] 791.12K  4.84MB/s    in 0.2s    

2019-12-29 11:55:20 (4.84 MB/s) - ‘chr1.GRCh38.excerpt.fasta’ saved [810105/810105]



In [8]:
# readGenome
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            if not line[0] == '>':
                genome += line.rstrip()
    return genome
t = readGenome('chr1.GRCh38.excerpt.fasta')
t[:200]



'TTGAATGCTGAAATCAGCAGGTAATATATGATAATAGAGAAAGCTATCCCGAAGGTGCATAGGTCAACAATACTTGAGCCTAACTCAGTAGATCCTAAAAGAAAGCAATTTTTGCTGCTAACCTAACATTTCACAATGTCTGGAGACATTTACAGTTCCCACAACCTATGGCAGTTACTGGCATCTACTAGAGGTCAGAG'

In [12]:
def editDistApproMatch(p, t):
    # Create distance matrix
    D = []
    for i in range(len(p)+1):
        D.append([0]*(len(t)+1))
    # Initialize first row of matrix
    for i in range(len(p)+1):
        D[i][0] = i
    # Fill in the rest of the matrix
    for i in range(1, len(p)+1):
        for j in range(1, len(t)+1):
            delt = 0 if p[i-1] == t[j-1] else 1
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            distDiag = D[i-1][j-1] + delt
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    return min(D[len(p)])




In [18]:
%%time
# check
p = 'GCGTATGC'
t = 'TATTGGCTATACGGTT'
editDistApproMatch(p,t)



CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 117 µs


2

In [26]:
# Q1: What is the edit distance of the best match between pattern GCTGATCGATCGTACGGCTGATCGATCGTACG
# and the excerpt of human chromosome 1? (Don't consider reverse complements.)
p = 'GCTGATCGATCGTACG'
t = readGenome('chr1.GRCh38.excerpt.fasta')




In [27]:
%%time
editDistApproMatch(p,t)




CPU times: user 8.27 s, sys: 36 ms, total: 8.31 s
Wall time: 8.32 s


3

In [28]:
# Q2: What is the edit distance of the best match between pattern GCTGATCGATCGTACGGCTGATCGATCGTACG
# and the excerpt of human chromosome 1? (Don't consider reverse complements.)
p = 'GATTTACCAGATTGAG'
t = readGenome('chr1.GRCh38.excerpt.fasta')





In [29]:
%%time
editDistApproMatch(p,t)




CPU times: user 8.27 s, sys: 48 ms, total: 8.32 s
Wall time: 8.34 s


2

In [ ]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match
        
        
        


In [30]:
# download sequence reads
!wget --no-check https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.for_asm.fastq




--2019-12-29 13:45:30--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.for_asm.fastq
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 13.35.112.129, 13.35.112.73, 13.35.112.183, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|13.35.112.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2562951 (2.4M) [application/octet-stream]
Saving to: ‘ERR266411_1.for_asm.fastq’

ERR266411_1.for_asm 100%[===================>]   2.44M  11.7MB/s    in 0.2s    

2019-12-29 13:45:31 (11.7 MB/s) - ‘ERR266411_1.for_asm.fastq’ saved [2562951/2562951]



In [1]:
# Read the fastq sequence
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()
            seq = fh.readline().rstrip()
            fh.readline()
            qual = fh.readline().rstrip()
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities




In [2]:
reads, quals = readFastq('ERR266411_1.for_asm.fastq')





In [3]:
reads[0]




'TAAACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAATATCCGAAAGTGTTAACTTCTGCGTCATGGAAGCGATAAAACTC'

In [4]:
# make index
import bisect
import sys
class Index(object):
    def __init__(self, t, k):
        ''' Create index from all substrings of size 'length' '''
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer
    
    def query(self, p):
        ''' Return index hits for first k-mer of P '''
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits





In [5]:
kMers = {}
for i in range(len(reads)):
    index = Index(reads[i], k = 30)
    for mer in index.index:
        if mer[0] in kMers.keys():
            kMers[mer[0]] = kMers[mer[0]] | set([i])
        else:
            kMers[mer[0]] = set([i])
        
        
        


In [6]:
kMersSortedIndex = sorted(kMers.keys())



In [7]:
kMersSortedIndex[-10:]




['TTTTTTGACGAGTAACAAAGTTTGGATTGC',
 'TTTTTTGACGAGTAACAAAGTTTGGATTGT',
 'TTTTTTGACGAGTTACACAGTTTGGATTGC',
 'TTTTTTGAGTGGTGCACTTATTCAAGAAAA',
 'TTTTTTGATAAAGCAGGAATTACTACTGCT',
 'TTTTTTGCCTGTTTGGTTCGCTTTGAGGAG',
 'TTTTTTGGCGAGGGACAGAGGGTGGGTTGG',
 'TTTTTTTCATGGTGGAGTTTCTCCTGTTGA',
 'TTTTTTTGAGTGGTGCACTTATTCAAGAAA',
 'TTTTTTTGATAAAGCAGGAATTACTACTGC']

In [8]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match
        





In [9]:
def kmer_overlap_map(reads, k):
    olaps = {}
    for b in range(len(reads)):
        if reads[b][:30] in kMers.keys():
            for a in kMers[reads[b][:30]]:
                if a == b:
                    continue
                olen = overlap(reads[a], reads[b], min_length=k)
                if olen > 0:
                    olaps[(a, b)] = olen
    return olaps





In [10]:
%%time
olaps = kmer_overlap_map(reads, k=30)




CPU times: user 4.94 s, sys: 76 ms, total: 5.02 s
Wall time: 5.04 s


In [12]:
olaps[:10]




{(2351, 2299): 65,
 (8826, 7670): 39,
 (4777, 5281): 58,
 (9971, 9055): 52,
 (6406, 7371): 45,
 (3367, 4411): 37,
 (2158, 2790): 55,
 (4067, 2888): 91,
 (4949, 4740): 92,
 (9293, 8567): 65,
 (7731, 9006): 38,
 (1213, 782): 71,
 (309, 204): 82,
 (2404, 1374): 34,
 (8654, 7382): 32,
 (7190, 7976): 53,
 (7732, 7226): 72,
 (1266, 2220): 95,
 (3369, 3997): 53,
 (8069, 7398): 64,
 (2048, 1262): 50,
 (7125, 8572): 82,
 (2022, 2815): 56,
 (3419, 2378): 32,
 (6621, 5611): 34,
 (5017, 7890): 50,
 (6008, 9037): 39,
 (3554, 2671): 44,
 (6540, 7178): 32,
 (7907, 7194): 59,
 (1950, 4091): 63,
 (22, 761): 50,
 (1788, 868): 43,
 (2788, 1251): 36,
 (2310, 1789): 64,
 (3830, 2907): 43,
 (8060, 8663): 33,
 (2857, 1910): 36,
 (1900, 2758): 43,
 (8945, 8213): 60,
 (8666, 9822): 47,
 (4998, 4283): 67,
 (4643, 6961): 41,
 (5884, 9016): 36,
 (6127, 5531): 57,
 (6769, 5768): 32,
 (6058, 3454): 36,
 (678, 39): 53,
 (240, 63): 60,
 (1072, 542): 57,
 (7615, 8046): 32,
 (268, 2067): 47,
 (3401, 5131): 79,
 (5640, 

In [13]:
len(olaps)




904746

In [19]:
keys = list(olaps.keys())




In [20]:
keys[:10]




[(2351, 2299),
 (8826, 7670),
 (4777, 5281),
 (9971, 9055),
 (6406, 7371),
 (3367, 4411),
 (2158, 2790),
 (4067, 2888),
 (4949, 4740),
 (9293, 8567),
 (7731, 9006),
 (1213, 782),
 (309, 204),
 (2404, 1374),
 (8654, 7382),
 (7190, 7976),
 (7732, 7226),
 (1266, 2220),
 (3369, 3997),
 (8069, 7398),
 (2048, 1262),
 (7125, 8572),
 (2022, 2815),
 (3419, 2378),
 (6621, 5611),
 (5017, 7890),
 (6008, 9037),
 (3554, 2671),
 (6540, 7178),
 (7907, 7194),
 (1950, 4091),
 (22, 761),
 (1788, 868),
 (2788, 1251),
 (2310, 1789),
 (3830, 2907),
 (8060, 8663),
 (2857, 1910),
 (1900, 2758),
 (8945, 8213),
 (8666, 9822),
 (4998, 4283),
 (4643, 6961),
 (5884, 9016),
 (6127, 5531),
 (6769, 5768),
 (6058, 3454),
 (678, 39),
 (240, 63),
 (1072, 542),
 (7615, 8046),
 (268, 2067),
 (3401, 5131),
 (5640, 4626),
 (8730, 9267),
 (3901, 4377),
 (9302, 8606),
 (5718, 5344),
 (5905, 4722),
 (8680, 7552),
 (2670, 3920),
 (2601, 1743),
 (6694, 5931),
 (7350, 6511),
 (8750, 9240),
 (6144, 2537),
 (9095, 8062),
 (2188, 15

In [22]:
listNodeSuffixes = set()
for i in range(len(keys)):
    node = keys[i][0]
    listNodeSuffixes.add(node)





In [24]:
len(listNodeSuffixes)




7161